First, let's import python modules that we'll need, everything is in
the standard library except for the astpretty module.  We're using
that to output an AST in a more readible format.  Feel free to replace
the ast.pprint calls with ast.dump calls if you want to avoid 
installing the astpretty module 






In [2]:
import argparse
import ast
import collections
import copy
import sys
import timeit

import astpretty

We'll use `example.py` as the source code that we'll examine and modify.  The `ast.parse` method will take a string with python code and return a corresponding `ast.AST` object.  We'll use this throughout the code to load and generate an AST from `example.py`

Now, let's look at using the `ast.NodeVisitor` to examine our source code.  First, we'll do the functional equivalent of a hello world program by printing out any functions that we define in our source code.  We use `ast.NodeVisitor` to do this by inheriting from the class and overriding the various visit_* methods.  I've done this in two different ways:

* override `generic_visit` and then if the node we're visiting is an `ast.FunctionDef` object, print out it's name
* override `visit_FunctionDef` and then print out the node name since this method only gets called on FunctionDef nodes

In [3]:
class HelloVisitor(ast.NodeVisitor):

  def generic_visit(self, node):
    if node.__class__ == ast.FunctionDef:
      sys.stdout.write(f"Defining function: {node.name} \n")
    super().generic_visit(node)


class FunctionVisitor(ast.NodeVisitor):

  def visit_FunctionDef(self, node):
    sys.stdout.write(f"Defining function: {node.name} \n")

Finally we'll instantiate our objects and call the `visit` methods on the objects to traverse the AST tree:

In [5]:
  with open("example.py", "r") as source:
    tree = ast.parse(source.read())
  sys.stdout.write("Visiting nodes using generic_visit:\n")
  hello_visitor = HelloVisitor()
  hello_visitor.visit(tree)
  sys.stdout.write("Visiting nodes using specific visit function:\n")
  function_visitor = FunctionVisitor()
  function_visitor.visit(tree)


Visiting nodes using generic_visit:
Defining function: test_function 
Defining function: test_function2 
Visiting nodes using specific visit function:
Defining function: test_function 
Defining function: test_function2 


Now we'll do something a bit more complex with the `NodeVisitor` class. We'll do a bit of static analysis to flag variables that are defined but haven't been used.  We'll do this by looking at all the `Assign` nodes to record all the variables created.  Next we'll look at the `Name` nodes to record when the variables are used.  By comparing the two lists, we can see when a variable has been created but hasn't been used.  

**Note:**  we need to be careful here to respect python's scoping rules.  We need to record if a variable is defined within a function so that we can catch cases where a variable is created in one scope and used in another scope. To make life (and the code) simpler, we'll ignore class definitions entirely but handling that would be analagous to handling variables within a function.  Also, this misses variables used in code that is evaluated at runtime, but if you're doing that, you should be prepared for this.

In [3]:
class UnusedVariables(ast.NodeVisitor):
  """
  Print out variables that are set but might be unused
  """

  def __init__(self):
    super().__init__()
    self.variable_def = {'global': {'global': [],
                                    'nonlocal': [],
                                    'local': []}}
    self.variable_used = {'global': {}}
    self.stack = ["global"]
    self.function_name = "global"

  def check(self, node: ast.AST) -> None:
    self.generic_visit(node)
    for var in self.variable_def['global']['local']:
      if var[0] not in self.variable_used['global']:
        sys.stdout.write(f"Variable {var[0]} defined in global on line {var[2]} not used\n")

  def visit_FunctionDef(self, node: ast.FunctionDef) -> None:
    self.function_name = node.name
    self.stack.append(node.name)
    self.variable_used[node.name] = {}
    self.variable_def[node.name] = {'global': [],
                                    'nonlocal': [],
                                    'local': []}
    for arg in node.args.args:
      self.variable_def[node.name]['local'].append((arg.arg, node.name, node.lineno))
    super().generic_visit(node)
    for var in self.variable_def[node.name]['local']:
      if var[0] not in self.variable_used[node.name]:
        sys.stdout.write(f"Variable {var[0]} defined in {node.name} on line {var[2]} not used\n")
    del self.variable_used[node.name]
    del self.variable_def[node.name]
    # remove function information
    self.function_name = self.stack.pop()
    self.function_name = self.stack[-1]

  def register_variable(self, scope: str, var_name: str, line: int) -> None:
    if scope in self.variable_def[self.function_name]:
      if var_name not in [x[0] for x in self.variable_def[self.function_name][scope]]:
        self.variable_def[self.function_name][scope].append((var_name, self.function_name, line))
    else:
      self.variable_def[self.function_name][scope] = [(var_name, self.function_name, line)]

  def visit_Nonlocal(self, node: ast.Nonlocal) -> None:
    for name in node.names:
      self.register_variable('nonlocal', name, node.lineno)

  def visit_Global(self, node: ast.Global) -> None:
    for name in node.names:
      self.register_variable('global', name, node.lineno)

  def visit_Assign(self, node: ast.Assign) -> None:
    # register the LHS of assignment
    if isinstance(node.targets[0], ast.Name):
      self.register_variable('local', node.targets[0].id, node.lineno)

    # examine the RHS of assignment
    super().generic_visit(node.value)

  def visit_Call(self, node: ast.Call) -> None:
    # visit parameters and register them
    for arg in node.args:
      if isinstance(arg, ast.Name):
        self.register_usage(arg.id)

  def var_defined(self, scope: str, var_name: str, function: str = None) -> bool:
    if function is None:
      function = self.function_name
    return var_name in [x[0] for x in self.variable_def[function][scope]]

  def register_usage(self, var_name: str) -> None:

    # if we know the var is global we can short-circuit the registration
    if self.var_defined('global', var_name):
      if var_name not in self.variable_used['global']:
        self.variable_used['global'][var_name] = True
        return

    # otherwise we need to check up the stack
    for function in reversed(self.stack):
      # short circuit global vars
      if self.var_defined('global', var_name, function):
        if var_name not in self.variable_used['global']:
          self.variable_used['global'][var_name] = True
          return

      # nonlocal variables are defined deeper in the stack
      if self.var_defined('nonlocal', var_name, function):
        continue

      # if definition found, mark var as being used and exit
      if self.var_defined('local', var_name, function):
        if var_name not in self.variable_used[function]:
          self.variable_used[function][var_name] = True
          return

      # continue going deeper in the stack to find the variable

    # if we're here, we haven't found the variable anywhere
    sys.stdout.write(f"In {self.function_name}, {var_name} used without being defined\n")

  def visit_Name(self, node: ast.Name) -> None:
    self.register_usage(node.id)


The basic idea of the code is to run some initialization call whenever a function is defined in order to setup dictionaries to track local, nonlocal, and global variables as well as a "stack" of where we are.  Then the code gets invoked whenever a `nonlocal`, `global`, function call, variable assignment, or variable reference is made. The code will then register the variable in the appropriate location in the `self.variable_def` dictionary. When a variable is used (i.e. gets referenced in a `Name` node), the `self.variable_used` dictionary gets updated.  Finally, once a function body is processed, the code will go through and see which variables are not used.

The code below runs this analysis on a test file(`var_assignment.py`)

In [4]:
with open("var_assignment.py", "r") as source:
    tree = ast.parse(source.read())
check_code = UnusedVariables()
check_code.check(tree)

Variable a defined in test_function2 on line 11 not used
Variable c defined in test_function2 on line 11 not used
In test_traversal, baz used without being defined
In global, __name__ used without being defined
Variable baz defined in global on line 40 not used
Variable unused defined in global on line 41 not used


Note that `__name__` is marked as being used without being defined since the code doesn't know about builtins.  Also, baz is incorrectly analyzed.  That's because the test_traversal function is defined before the code for file.  This can be avoided by having the code do multiple analysis passes to catch forward references like this.

### Static Analysis

Now we'll look at using static analysis to modify ASTs and possibly optimize our code.  We'll do two different optimizations:

* dead code elimination
* function inlining

#### Dead code elimination

Dead code elimination involves analyzing our code and removing code that we can guarantee will never run.  For example, in the following code: 

```
  if True:
    a = 5
  else:
    a = 10
```

we know that the ```a=5``` branch will always be taken.  So we can effectively replace a comparison and variable assignment with a single assignment.  This is potentially a next bonus by reducing our code size, and avoiding a comparison and jump.

This becomes even more powerful if we use constant folding and constant propagation to evaluate code at runtime. E.g. replacing code like

```
x = 5
y = x + 10
if y > 5:
  x = 20
```

with 

```
x = 5
y = 15
if 15 > 5:
  x = 20
```
then
```
x = 5
y = 15
x = 20
```

and finally 

```
y = 15
x = 20
```

Notice that in optimizing the code, running some optimizations like dead code elimination at different stages might be useful.

#### Function inlining

Function inlining is a bit more ambiguous optimization.  Function inlining involves taking the body of a function, and replacing function calls with the body of the function.  E.g. `a = func(a)` gets replaced with the code for `func` and instead of having `func` return a value, it is assigned to `a`.  Depending on the size of the function, this may be a net benefit or may even slow down the code.  This also is a bit more complicated since we need to propagate function arguments into the body and make sure that variables declared within the function are renamed so that they don't overwrite variables where the function was called.  

To make the code much simpler, we'll just inline functions that consist of calls to other functions. 

First we'll define a helper function to time our code fragments. The `timeit.timeit` call runs our code fragment 10,000 times and outputs the time it took to run.  Since timeit runs the code in a clean namespace we need to use the globals keyword arg to pass in the AST that we want to time and then use the `compile` and `exec` calls to first compile an `ast.AST` object to executable code and then to execute it.

In [5]:
def time_tree(tree: ast.AST) -> float:
  """
  Time ast execution

  :param tree: AST tree
  :return: float recording number of seconds to run snippet
  """
  return timeit.timeit(stmt="exec(compile(tree, 'test', mode='exec'))", number=10000, globals={'tree': tree})

#### Dead code elimination using ASTs

Now, we'll look at the code for doing dead code elimination:

In [6]:
class RemoveDeadCode(ast.NodeTransformer):
  """
  Simplify branches that test constants to remove dead code
  e.g. if true: a = 5 else a = 10  gets turned into a = 5
  """

  def visit_If(self, node):
    # make sure to recurse on branches
    super().generic_visit(node)

    if type(node.test) in [ast.Constant, ast.NameConstant]:
      if node.test.value:
        new_node = node.body
      else:
        new_node = node.orelse

      if not new_node:
        # if branch being used is empty, delete
        return None
      return new_node
    else:
      return node

  def optimize(self, node: ast.AST):
    """
    Optimize nodes

    :param node: node to optimize
    :return: ast tree
    """
    new_node = self.visit(node)
    ast.fix_missing_locations(new_node)
    return new_node


The code is fairly simple, it looks at any if statements and if the comparison in the if statement is a constant, the code will evaluate the constant and return the code in the appropriate branch.  There's quite a bit more that can be done but that's left as an exercise for the reader.

#### Function inlining

As mentioned before function inlining is quite a bit more complex since we need to be careful to avoid overwriting variables and to preserve scoping rules when moving code around.  To make things much more simple, we'll only inline functions where the body consists of other function calls and which does not return any values.  E.g. a function like this

```
def foo(a, b, c):
   bar(a)
   baz(a, c)
   bar(b)
   
```

This will let us doing inlining by replacing the function call with the body after some substitution of function arguments.

First, we'll write some code that'll scan the source and figure out which functions can be inlined and to cache the function body if a function can be inlined.

In [7]:
class CallOnlyChecker(ast.NodeVisitor):
  """
  Visit and check function definitions to make sure that it can be inlined,
  i.e. make sure function only consists of calls

  Also cache function defs so that they can later be inlined
  """

  def __init__(self):
    self.__function_cache__ = collections.defaultdict(lambda: {'args': [], 'body': []})
    self.__inlineable_cache__ = collections.defaultdict(lambda: False)
    super().__init__()

  def can_inline(self, name: str) -> bool:
    """
    Check to see if function can be inlined

    :param name: name of function to check
    :return: True if function can be inlined, False otherwise
    """
    return self.__inlineable_cache__[name]

  def get_cached_function(self, name: str) -> dict:
    """
    Get dictionary with function information

    :param name: name of function
    :return: a
    """
    return copy.copy(self.__function_cache__[name])

  def visit_FunctionDef(self, node: ast.FunctionDef):
    """
    Examine function and cache if needed
    :param node: ast.FunctionDef to analyze
    :return: None
    """
    for func_node in ast.iter_child_nodes(node):
      if isinstance(func_node, ast.Expr) and type(func_node.value) in [ast.Str, ast.Call]:
        next
      elif isinstance(func_node, ast.arguments):
        next
      else:
        self.__inlineable_cache__[node.name] = False
        return
    self.__inlineable_cache__[node.name] = True
    self.__function_cache__[node.name] = {'args': node.args}
    if isinstance(node.body[0], ast.Expr) and isinstance(node.body[0].value, ast.Str):
      # skip docstring
       call_list = node.body[1:]
    else:
      call_list = node.body
    call_list = map(lambda x: x.value, call_list)
    self.__function_cache__[node.name]['body'] = call_list


The code just examines all the `FunctionDef` nodes and then looks at the function body to see if it can be inlined and caches the function body if that's the case. 

Now the code that does the actual inlining:

In [8]:
class InlineFunctions(ast.NodeTransformer):
  """Inline functions that only have calls in them"""

  def __init__(self):
    self.checker = CallOnlyChecker()
    super().__init__()

  def optimize(self, tree: ast.AST):
    """
    verify and inline functions if functions only consist of calls
    :param tree: ast node
    :return: None
    """

    # get information about ast
    self.checker.visit(tree)
    inlined_tree = self.visit(tree)
    inlined_tree = CleanupAST().cleanup(inlined_tree)
    ast.fix_missing_locations(inlined_tree)
    return inlined_tree

  def visit_Call(self, node):
    """
    Look at function calls and replace if possible
    """
    super().generic_visit(node)
    if self.checker.can_inline(node.func.id):
      replacement_code = self.checker.get_cached_function(node.func.id)
      replacement_args = replacement_code['args']
      replacement_nodes = []
      replacement_table = {}
      index = 0
      for arg in node.args:
        arg_name = replacement_args.args[index].arg
        replacement_table[arg_name] = arg
        index += 1
      for call in replacement_code['body']:
        new_args = []
        for arg in call.args:
          if isinstance(arg, ast.Name):
            if arg.id in replacement_table:
              new_args.append(replacement_table[arg.id])
            else:
              new_args.append(arg)
          else:
            new_args.append(arg)
        call.args = new_args
        replacement_nodes.append(call)
      if len(replacement_nodes) == 1:
        return replacement_nodes[0]
      else:
        for node in replacement_nodes:
          node.lineno = 0

      return replacement_nodes
    else:
      return node

This code just goes through an AST and first processes it to find functions that can be inlined.  Once that's done, it goes through the AST again and replaces a `Call` node with the appropriate `Call` nodes from within the function while doing substitutions between the parameters in the original `Call` node and the parameters in the function definition.  There's a few corner cases this code won't cover but the existing code should illustrate the general principles

There is one tricky bit here due to the way the python ASTs and the ast.NodeTransformer works.  The resulting AST will have `Expr` nodes with multiple `Call` nodes which is not allowed.  The following code will fix this by going through the AST and splitting these invalid `Expr` nodes into multiple `Expr` nodes.  

In [ ]:

class CleanupAST(ast.NodeTransformer):
  """
  Scan and cleanup ASTs to split Expr nodes with values that contain multiple
  expressions
  """

  def cleanBody(self, node_body: list) -> list:
    """
    Clean up expr nodes in a list, splitting expr with
    multiple statements in value field

    :param node_body: list of ast nodes
    :return: list of cleaned ast nodes
    """
    new_body = []
    for child_node in node_body:
      if isinstance(child_node, ast.Expr) and isinstance(child_node.value, list):
          if len(child_node.value) == 1:
            new_body.append(ast.Expr(value=child_node.value[0]))
          else:
            for stmt in child_node.value:
              new_body.append(ast.Expr(value=child_node.value[0]))
      else:
        new_body.append(child_node)
    return new_body

  def visit_Module(self, node: ast.Module):
    """
    Clean up expr nodes in a module

    :param node: ast.Module instance to cleanup
    :return: cleaned up node
    """
    super().generic_visit(node)

    node.body = self.cleanBody(node.body)
    return node

  def visit_FunctionDef(self, node: ast.FunctionDef):
    """
    Examine function and clean up if necessary
    :param node: ast.FunctionDef to analyze
    :return: None
    """
    super().generic_visit(node)

    node.body = self.cleanBody(node.body)
    return node

  def visit_If(self, node: ast.If):
    """
    Examine if statement and clean up if necessary
    :param node: ast.FunctionDef to analyze
    :return: None
    """
    super().generic_visit(node)

    node.body = self.cleanBody(node.body)
    node.orelse = self.cleanBody(node.orelse)
    return node

  def visit_For(self, node: ast.For):
    """
    Examine if statement and clean up if necessary
    :param node: ast.FunctionDef to analyze
    :return: None
    """
    super().generic_visit(node)

    node.body = self.cleanBody(node.body)
    node.orelse = self.cleanBody(node.orelse)
    return node

  def visit_While(self, node: ast.While):
    """
    Examine if statement and clean up if necessary
    :param node: ast.FunctionDef to analyze
    :return: None
    """
    super().generic_visit(node)

    node.body = self.cleanBody(node.body)
    node.orelse = self.cleanBody(node.orelse)
    return node

  def visit_With(self, node: ast.With):
    """
    Examine with statement
    :param node: ast.FunctionDef to analyze
    :return: None
    """
    super().generic_visit(node)

    node.body = self.cleanBody(node.body)
    return node

  def visit_Try(self, node: ast.Try):
    """
    Examine try statement
    :param node: ast.FunctionDef to analyze
    :return: None
    """
    super().generic_visit(node)

    node.body = self.cleanBody(node.body)
    node.finalbody = self.cleanBody(node.finalbody)
    return node

  def visit_TryExcept(self, node: ast.Try):
    """
    Examine try statement
    :param node: ast.FunctionDef to analyze
    :return: None
    """
    super().generic_visit(node)

    node.body = self.cleanBody(node.body)
    node.orelse = self.cleanBody(node.orelse)
    return node

  def cleanup(self, tree: ast.AST) -> ast.AST:
    """
    Clean up an AST tree, splitting incorrect Expr if found

    :param tree: AST to clean
    :return: cleaned AST
    """
    new_tree = self.visit(tree)
    ast.fix_missing_locations(new_tree)
    return new_tree

Now, let's see the code in action and see how it affects code execution time:

In [ ]:
  with open("example.py", "r") as source:
    tree = ast.parse(source.read())

  sys.stdout.write(f"AST  code:\n")
  astpretty.pprint(tree)
  code_timing = time_tree(tree)
  branch_transformer = RemoveDeadCode()
  pruned_tree = branch_transformer.optimize(tree)
  deadcode_timing = time_tree(pruned_tree)
  sys.stdout.write(f"transformed AST  code:\n")
  astpretty.pprint(pruned_tree)

  function_transformer = InlineFunctions()

  inlined_tree = function_transformer.optimize(pruned_tree)
  astpretty.pprint(inlined_tree)

  inlined_tree = ast.fix_missing_locations(inlined_tree)
  inlined_code_timing = time_tree(inlined_tree)
  sys.stdout.write(f"inlined AST  code:\n")
  astpretty.pprint(inlined_tree)


In [ ]:
  sys.stdout.write(f"Time for code: {code_timing}\n")
  sys.stdout.write(f"Time for code after using RemoveDeadCode: {deadcode_timing}\n")
  sys.stdout.write(f"Time for code after using RemoveDeadCode and InlineFunction: {inlined_code_timing}\n")

So the dead code eliminiation sped things up a bunch and inlining didn't do much.  These numbers will probably vary a bit on different systems but the relationship should remain about the same.

In [9]:

class CleanupAST(ast.NodeTransformer):
  """
  Scan and cleanup ASTs to split Expr nodes with values that contain multiple
  expressions
  """

  def cleanBody(self, node_body: list) -> list:
    """
    Clean up expr nodes in a list, splitting expr with
    multiple statements in value field

    :param node_body: list of ast nodes
    :return: list of cleaned ast nodes
    """
    new_body = []
    for child_node in node_body:
      if isinstance(child_node, ast.Expr) and isinstance(child_node.value, list):
          if len(child_node.value) == 1:
            new_body.append(ast.Expr(value=child_node.value[0]))
          else:
            for stmt in child_node.value:
              new_body.append(ast.Expr(value=child_node.value[0]))
      else:
        new_body.append(child_node)
    return new_body

  def visit_Module(self, node: ast.Module):
    """
    Clean up expr nodes in a module

    :param node: ast.Module instance to cleanup
    :return: cleaned up node
    """
    super().generic_visit(node)

    node.body = self.cleanBody(node.body)
    return node

  def visit_FunctionDef(self, node: ast.FunctionDef):
    """
    Examine function and clean up if necessary
    :param node: ast.FunctionDef to analyze
    :return: None
    """
    super().generic_visit(node)

    node.body = self.cleanBody(node.body)
    return node

  def visit_If(self, node: ast.If):
    """
    Examine if statement and clean up if necessary
    :param node: ast.FunctionDef to analyze
    :return: None
    """
    super().generic_visit(node)

    node.body = self.cleanBody(node.body)
    node.orelse = self.cleanBody(node.orelse)
    return node

  def visit_For(self, node: ast.For):
    """
    Examine if statement and clean up if necessary
    :param node: ast.FunctionDef to analyze
    :return: None
    """
    super().generic_visit(node)

    node.body = self.cleanBody(node.body)
    node.orelse = self.cleanBody(node.orelse)
    return node

  def visit_While(self, node: ast.While):
    """
    Examine if statement and clean up if necessary
    :param node: ast.FunctionDef to analyze
    :return: None
    """
    super().generic_visit(node)

    node.body = self.cleanBody(node.body)
    node.orelse = self.cleanBody(node.orelse)
    return node

  def visit_With(self, node: ast.With):
    """
    Examine with statement
    :param node: ast.FunctionDef to analyze
    :return: None
    """
    super().generic_visit(node)

    node.body = self.cleanBody(node.body)
    return node

  def visit_Try(self, node: ast.Try):
    """
    Examine try statement
    :param node: ast.FunctionDef to analyze
    :return: None
    """
    super().generic_visit(node)

    node.body = self.cleanBody(node.body)
    node.finalbody = self.cleanBody(node.finalbody)
    return node

  def visit_TryExcept(self, node: ast.Try):
    """
    Examine try statement
    :param node: ast.FunctionDef to analyze
    :return: None
    """
    super().generic_visit(node)

    node.body = self.cleanBody(node.body)
    node.orelse = self.cleanBody(node.orelse)
    return node

  def cleanup(self, tree: ast.AST) -> ast.AST:
    """
    Clean up an AST tree, splitting incorrect Expr if found

    :param tree: AST to clean
    :return: cleaned AST
    """
    new_tree = self.visit(tree)
    ast.fix_missing_locations(new_tree)
    return new_tree

Now, let's see the code in action and see how it affects code execution time:

In [13]:
  with open("example.py", "r") as source:
    tree = ast.parse(source.read())

  sys.stdout.write(f"AST  code:\n")
  astpretty.pprint(tree)
  code_timing = time_tree(tree)
  branch_transformer = RemoveDeadCode()
  pruned_tree = branch_transformer.optimize(tree)
  deadcode_timing = time_tree(pruned_tree)
  sys.stdout.write(f"transformed AST  code:\n")
  astpretty.pprint(pruned_tree)

  function_transformer = InlineFunctions()

  inlined_tree = function_transformer.optimize(pruned_tree)
  astpretty.pprint(inlined_tree)

  inlined_tree = ast.fix_missing_locations(inlined_tree)
  inlined_code_timing = time_tree(inlined_tree)
  sys.stdout.write(f"inlined AST  code:\n")
  astpretty.pprint(inlined_tree)


AST  code:
Module(
    body=[
        FunctionDef(
            lineno=4,
            col_offset=0,
            end_lineno=8,
            end_col_offset=13,
            name='test_function',
            args=arguments(posonlyargs=[], args=[], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]),
            body=[
                Expr(
                    lineno=5,
                    col_offset=2,
                    end_lineno=5,
                    end_col_offset=22,
                    value=Constant(lineno=5, col_offset=2, end_lineno=5, end_col_offset=22, value='Docstring test', kind=None),
                ),
                Expr(
                    lineno=6,
                    col_offset=2,
                    end_lineno=6,
                    end_col_offset=10,
                    value=Call(
                        lineno=6,
                        col_offset=2,
                        end_lineno=6,
                        end_col_offset=10,
                   

transformed AST  code:
Module(
    body=[
        FunctionDef(
            lineno=4,
            col_offset=0,
            end_lineno=8,
            end_col_offset=13,
            name='test_function',
            args=arguments(posonlyargs=[], args=[], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]),
            body=[
                Expr(
                    lineno=5,
                    col_offset=2,
                    end_lineno=5,
                    end_col_offset=22,
                    value=Constant(lineno=5, col_offset=2, end_lineno=5, end_col_offset=22, value='Docstring test', kind=None),
                ),
                Expr(
                    lineno=6,
                    col_offset=2,
                    end_lineno=6,
                    end_col_offset=10,
                    value=Call(
                        lineno=6,
                        col_offset=2,
                        end_lineno=6,
                        end_col_offset=10,
       

inlined AST  code:
Module(
    body=[
        FunctionDef(
            lineno=4,
            col_offset=0,
            end_lineno=8,
            end_col_offset=13,
            name='test_function',
            args=arguments(posonlyargs=[], args=[], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]),
            body=[
                Expr(
                    lineno=5,
                    col_offset=2,
                    end_lineno=5,
                    end_col_offset=22,
                    value=Constant(lineno=5, col_offset=2, end_lineno=5, end_col_offset=22, value='Docstring test', kind=None),
                ),
                Expr(
                    lineno=6,
                    col_offset=2,
                    end_lineno=6,
                    end_col_offset=10,
                    value=Call(
                        lineno=6,
                        col_offset=2,
                        end_lineno=6,
                        end_col_offset=10,
           

In [12]:
  sys.stdout.write(f"Time for code: {code_timing}\n")
  sys.stdout.write(f"Time for code after using RemoveDeadCode: {deadcode_timing}\n")
  sys.stdout.write(f"Time for code after using RemoveDeadCode and InlineFunction: {inlined_code_timing}\n")

Time for code: 0.9266150880002897
Time for code after using RemoveDeadCode: 0.6605467760000465
Time for code after using RemoveDeadCode and InlineFunction: 0.6588795940001546


So the dead code eliminiation sped things up a bunch and inlining didn't do much.  These numbers will probably vary a bit on different systems but the relationship should remain about the same.